In [1]:
# import os
# import zipfile
# import shutil
# import yaml
# from pathlib import Path

# def extract_and_merge_datasets(zip_path, output_dir="merged_dataset"):

#     # Create temporary directory for extraction
#     temp_dir = "temp_extract"
#     os.makedirs(temp_dir, exist_ok=True)

#     # Create output directory structure
#     os.makedirs(output_dir, exist_ok=True)
#     images_dir = os.path.join(output_dir, "images")
#     labels_dir = os.path.join(output_dir, "labels")
#     os.makedirs(images_dir, exist_ok=True)
#     os.makedirs(labels_dir, exist_ok=True)

#     # Extract the zip file
#     print(f"Extracting {zip_path} to {temp_dir}...")
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         zip_ref.extractall(temp_dir)

#     # Look for data.yaml in the extracted files
#     yaml_path = None
#     class_names = []
#     nc = 0

#     for root, dirs, files in os.walk(temp_dir):
#         for file in files:
#             if file == "data.yaml":
#                 yaml_path = os.path.join(root, file)
#                 break

#     # Read class information if data.yaml exists
#     if yaml_path:
#         with open(yaml_path, 'r') as f:
#             yaml_data = yaml.safe_load(f)
#             if 'names' in yaml_data:
#                 class_names = yaml_data['names']
#                 nc = len(class_names)
#             elif 'classes' in yaml_data and isinstance(yaml_data['classes'], list):
#                 class_names = yaml_data['classes']
#                 nc = len(class_names)

#     # Find and merge the train, val, test directories
#     for split in ['train', 'valid', 'test']:
#       split_dir = os.path.join(temp_dir, split)

#       if os.path.exists(split_dir):
#           print(f"Processing {split} directory...")

#           # ✅ 들여쓰기 하나만 (if 블록 내부)
#           split_images_dir = os.path.join(split_dir, 'images')
#           split_labels_dir = os.path.join(split_dir, 'labels')

#           if os.path.exists(split_images_dir):
#               for img in os.listdir(split_images_dir):
#                   src = os.path.join(split_images_dir, img)
#                   dst = os.path.join(images_dir, img)
#                   os.makedirs(os.path.dirname(dst), exist_ok=True)
#                   shutil.copy2(src, dst)

#           if os.path.exists(split_labels_dir):
#               for ann in os.listdir(split_labels_dir):
#                   src = os.path.join(split_labels_dir, ann)
#                   dst = os.path.join(labels_dir, ann)
#                   os.makedirs(os.path.dirname(dst), exist_ok=True)
#                   shutil.copy2(src, dst)

#     # Create new data.yaml file
#     img_count = len(os.listdir(images_dir))
#     label_count = len(os.listdir(labels_dir))

#     # Determine path to dataset
#     dataset_path = os.path.abspath(output_dir)

#     # Create new data.yaml
#     data_yaml = {
#         'path': dataset_path,
#         'train': 'images',
#         'val': 'images',
#         'test': 'images',
#         'nc': nc,
#         'names': class_names
#     }

#     yaml_out_path = os.path.join(output_dir, 'data.yaml')
#     with open(yaml_out_path, 'w') as f:
#         yaml.dump(data_yaml, f, default_flow_style=False, sort_keys=False)

#     # Clean up temporary directory
#     print(f"Cleaning up temporary files...")
#     shutil.rmtree(temp_dir)

#     print(f"Successfully merged datasets into {output_dir}")
#     print(f"Total images: {img_count}")
#     print(f"Total labels: {label_count}")
#     print(f"Created data.yaml with {nc} classes")
#     if nc > 0:
#         print(f"Classes: {', '.join(class_names)}")
#     else:
#         print("Warning: No class information found in original data.yaml")
# #

In [2]:
# zip_files= ['/content/drive/MyDrive/CV2/Datasets/final-airplane--1.zip']
# for zip_path in zip_files:
#     output_dir = zip_path.split('.')[0]
#     print(f"Processing {zip_path} -> {output_dir}")
#     print("="*50)
#     extract_and_merge_datasets(zip_path, output_dir)
#     print("\n")

In [3]:
# import os
# import random
# import shutil
# import yaml
# from glob import glob
# from sklearn.model_selection import train_test_split

# def create_random_splits(data_dir, n_splits=5, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2):
#     """
#     단일 데이터 경로에서 이미지와 라벨 데이터셋을 N개의 랜덤 train/val/test로 분할하고 관련 파일을 생성합니다. #왜냐면 한개로 합쳐서

#     Args:
#         data_dir (str): 데이터 디렉토리 경로 (images, labels 폴더와 data.yaml 파일을 포함)
#         n_splits (int): 생성할 데이터셋 분할 수
#         train_ratio (float): 학습 데이터 비율
#         val_ratio (float): 검증 데이터 비율
#         test_ratio (float): 테스트 데이터 비율
#     """
#     # 비율 합이 1인지 확인
#     assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-10, "비율의 합은 1이어야 합니다" #assert제대로 안쓰면 error배출 true일때는 넘어감

#     # 경로 설정
#     images_dir = os.path.join(data_dir, 'images')
#     labels_dir = os.path.join(data_dir, 'labels')
#     yaml_path = os.path.join(data_dir, 'data.yaml')

#     # 디렉토리 존재 확인
#     if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
#         print(f"경고: {data_dir}에 images 또는 labels 폴더가 없습니다. 건너뜁니다.")
#         return

#     # 이미지 파일 목록 가져오기 (확장자는 필요에 따라 수정)
#     image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
#     image_files = []
#     for ext in image_extensions:
#         image_files.extend(glob(os.path.join(images_dir, ext))) #glob() 함수는 인자로 받은 패턴과 이름이 일치하는 모든 파일과 디렉터리의 리스트를 반환
#     #즉 저기에 있는 확장자들은 다 return해준다고 생각하면면됨됨
#     # 이미지 파일명만 추출 (확장자 포함)
#     image_filenames = [os.path.basename(f) for f in image_files] #자기 위치 폴더면 폴더명명 파일이면 파일명 반환환

#     # 해당 이미지 파일에 대응하는 라벨 파일이 있는지 확인
#     valid_samples = []
#     for img_filename in image_filenames:
#         # 이미지 파일명에서 확장자를 제거하고 라벨 파일 확장자 추가
#         base_name = os.path.splitext(img_filename)[0]
#         label_path = os.path.join(labels_dir, base_name + '.txt')

#         # 라벨 파일이 존재하면 유효한 샘플로 간주
#         if os.path.exists(label_path):
#             valid_samples.append((img_filename, base_name + '.txt')) #일단 라벨을 다 만들었는데 있으면 이게 실행이됨 그래서 이미지랑 라벨이 둘다 있으니까 valid sample에 추가가

#     if not valid_samples:
#         print(f"오류: {data_dir}에서 유효한 샘플을 찾을 수 없습니다.")
#         return
#     #여기까지 공부부

#     # data.yaml 파일 불러오기
#     try:
#         with open(yaml_path, 'r') as f:
#             data_yaml = yaml.safe_load(f)
#             nc = data_yaml.get('nc', 0)
#             names = data_yaml.get('names', [])
#     except FileNotFoundError:
#         print(f"경고: {yaml_path} 파일을 찾을 수 없습니다. nc와 names는 기본값으로 설정됩니다.")
#         nc = 0
#         names = []

#     print(f"{data_dir}: {len(valid_samples)}개의 유효한 샘플을 찾았습니다.")

#     # N개의 랜덤 분할 생성
#     for iter_idx in range(1, n_splits + 1):
#         # 데이터 세트 분할
#         random.shuffle(valid_samples) #리스트에 있는 random sample을 알아서 나눔


#         # 먼저 train과 temp(val+test) 분할
#         train_samples, temp_samples = train_test_split(
#             valid_samples,
#             train_size=train_ratio,
#             test_size=val_ratio + test_ratio,
#             random_state=42 + iter_idx  # 각 반복마다 다른 시드 사용
#         )

#         # temp를 val과 test로 분할
#         val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
#         val_samples, test_samples = train_test_split(
#             temp_samples,
#             train_size=val_ratio_adjusted,
#             test_size=1 - val_ratio_adjusted,
#             random_state=42 + iter_idx
#         )

#         # 텍스트 파일 생성
#         train_txt_path = os.path.join(data_dir, f'train_iter_{iter_idx:02d}.txt')
#         val_txt_path = os.path.join(data_dir, f'val_iter_{iter_idx:02d}.txt')
#         test_txt_path = os.path.join(data_dir, f'test_iter_{iter_idx:02d}.txt')

#         # 학습 데이터 파일 쓰기
#         with open(train_txt_path, 'w') as f:
#             for img_file, _ in train_samples:
#                 img_path = os.path.join('images', img_file)  # 상대 경로 사용
#                 f.write(f"{img_path}\n")

#         # 검증 데이터 파일 쓰기
#         with open(val_txt_path, 'w') as f:
#             for img_file, _ in val_samples:
#                 img_path = os.path.join('images', img_file)  # 상대 경로 사용
#                 f.write(f"{img_path}\n")

#         # 테스트 데이터 파일 쓰기
#         with open(test_txt_path, 'w') as f:
#             for img_file, _ in test_samples:
#                 img_path = os.path.join('images', img_file)  # 상대 경로 사용
#                 f.write(f"{img_path}\n")

#         # YAML 파일 생성
#         output_yaml_path = os.path.join(data_dir, f'data_iter_{iter_idx:02d}.yaml')
#         yaml_content = {
#             'train': f'train_iter_{iter_idx:02d}.txt',  # 상대 경로 사용
#             'val': f'val_iter_{iter_idx:02d}.txt',      # 상대 경로 사용
#             'test': f'test_iter_{iter_idx:02d}.txt',    # 상대 경로 사용
#             'nc': nc,
#             'names': names
#         }

#         with open(output_yaml_path, 'w') as f:
#             yaml.dump(yaml_content, f, default_flow_style=False)

#         print(f"{data_dir} - Iteration {iter_idx} 완료: {len(train_samples)} 학습, {len(val_samples)} 검증, {len(test_samples)} 테스트")

In [4]:
# data_dirs = ['/content/drive/MyDrive/CV2/Datasets/final-airplane--1']
# n_splits = 10
# train_ratio = 0.6
# val_ratio = 0.2
# test_ratio = 0.2

# for data_dir in data_dirs:
#         print(f"\n처리 중: {data_dir}")
#         create_random_splits(data_dir, n_splits, train_ratio, val_ratio, test_ratio) #여기까지 data전처지지

In [5]:
# import os
# import cv2
# import albumentations as A
# from tqdm import tqdm

# # 설정
# txt_base_dir = '/content/drive/MyDrive/01/Datasets/final-airplane--1' #로컬에 맞춰서 개선
# input_img_dir = os.path.join(txt_base_dir, 'images')
# input_lbl_dir = os.path.join(txt_base_dir, 'labels')
# output_img_dir = input_img_dir
# output_lbl_dir = input_lbl_dir

# # 증강 정의
# transform = A.Compose([
#     A.RandomSizedCrop(min_max_height=(300, 500), size=(640, 640), p=0.3),
#     A.Rotate(limit=30, p=0.4),
#     A.HorizontalFlip(p=0.5),
#     A.HueSaturationValue(p=0.4),
#     A.RGBShift(p=0.3),
#     A.RandomBrightnessContrast(brightness_limit=(-0.4, 0.1), contrast_limit=0.3, p=0.4),
#     A.Resize(height=640, width=640, p=1.0),
#     A.GaussianBlur(p=0.1)
# ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.3))


# # train_iter_01 ~ train_iter_10
# for i in range(1, 11):
#     iter_id = f"{i:02d}"
#     txt_name = f"train_iter_{iter_id}.txt"
#     txt_path = os.path.join(txt_base_dir, txt_name)

#     if not os.path.exists(txt_path):
#         print(f"❌ {txt_name} 없음 - 건너뜀")
#         continue

#     with open(txt_path, 'r') as f:
#         image_paths = [line.strip() for line in f.readlines()]

#     print(f"🔄 {txt_name} 처리 중 ({len(image_paths)}장)")

#     new_lines = []

#     for img_path in tqdm(image_paths):
#         img_name = os.path.basename(img_path)
#         img_stem = os.path.splitext(img_name)[0]
#         input_img_path = os.path.join(input_img_dir, img_name)
#         input_lbl_path = os.path.join(input_lbl_dir, f"{img_stem}.txt")

#         image = cv2.imread(input_img_path)
#         if image is None:
#             print(f"❌ 이미지 로드 실패: {input_img_path}")
#             continue

#         # 라벨 로드
#         bboxes = []
#         class_labels = []
#         if os.path.exists(input_lbl_path):
#             with open(input_lbl_path, 'r') as f:
#                 for line in f:
#                     parts = line.strip().split()
#                     if len(parts) == 5:
#                         cls, x, y, w, h = parts
#                         bboxes.append([float(x), float(y), float(w), float(h)])
#                         class_labels.append(int(cls))

#         try:
#             transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
#         except Exception as e:
#             print(f"⚠️ bbox 오류로 해당 이미지 건너뜀: {img_path}")
#             continue

#         aug_image = transformed['image']
#         aug_bboxes = transformed['bboxes']
#         aug_labels = transformed['class_labels']

#         if not aug_bboxes:
#             continue

#         # 저장 파일명
#         aug_img_name = f"aug_{img_name}"
#         aug_lbl_name = f"aug_{img_stem}.txt"
#         save_img_path = os.path.join(output_img_dir, aug_img_name)
#         save_lbl_path = os.path.join(output_lbl_dir, aug_lbl_name)

#         # 저장
#         cv2.imwrite(save_img_path, aug_image)
#         with open(save_lbl_path, 'w') as f:
#             for bbox, cls in zip(aug_bboxes, aug_labels):
#                 f.write(f"{cls} {' '.join(f'{x:.6f}' for x in bbox)}\n")

#         # txt에 기록할 새로운 이미지 경로
#         new_lines.append(save_img_path)

#     # 기존 train_iter_xx.txt에 덧붙이기
#     with open(txt_path, 'a') as f:
#         for new_line in new_lines:
#             f.write(f"{new_line}\n")

#     print(f"✅ {txt_name} 에 증강 이미지 {len(new_lines)}장 추가 완료")

# print("🎉 전체 train_iter_xx 파일에 증강 이미지 경로 추가 완료")
#이건 전체과정 한번에 다한거임

In [ ]:
from competition_utils import *
# submission_1_20214173.py 상단에 추가!
from models.common import CBAM  # common.py에 정의한 CBAM 경로

# 등록: YAML에서 'CBAM'이라는 이름으로 쓸 수 있도록 연결
import ultralytics.nn.modules as modules
modules.CBAM = CBAM

# 제출 함수를 리스트로 정의 (submission_N_학번)
# 깃허브에는 최대 3개 업로드 데모시 하이퍼파라미터 튜닝 후 1개만 제출
submission_functions = ['submission_1_20214173', 'submission_2_20214173', 'submission_3_20214173']  
for submission_function in submission_functions:
    exec(f"from {submission_function} import {submission_function}")

# 분석 방향에 따라 iteration 수 수정
# 평가시에는 데모와 다른 스플릿 시드를 만들어 [1, 10]로 진행 예정
iterations = [1, 10] 

# 'Crown Detection'는 예제 데이터셋, 데모 및 평가시에는 'CV_Competition'
Dataset_Name = 'Crown Detection' 

# 결과를 모아 하나의 csv 파일에 저장
results_df = pd.DataFrame(columns=[
    'Experiment Time', 'Iteration', 'Submission Function', 
    'IoU', 'Dice', 'Precision', 'Recall', 'Output Json Path',
])
csv_filename = f"Evaluation_Results_{datetime.now().strftime('%y%m%d_%H%M%S')}.csv"

# 서로 다른 iteration, submission function으로 실험 진행
for iteration in range(iterations[0], iterations[1] + 1):
    yaml_path = f'Datasets/{Dataset_Name}/data_iter_{iteration:02d}.yaml'
    for submission_function in submission_functions:
        ex_time = datetime.now().strftime('%y%m%d_%H%M%S')
        output_json_path = f"{ex_time}_{submission_function}_Iter_{iteration}_detection_results.json"
        globals()[submission_function](yaml_path, output_json_path)
        labels_dir = f"Datasets/{Dataset_Name}/labels"  
        vis_output_dir = f"{ex_time}_visualization_results"  
        image_level_result_path =f"{ex_time}_{submission_function}_Iter_{iteration}_image_level_results.json"
        stats = eval_and_vis(yaml_path, output_json_path, labels_dir, image_level_result_path, vis_output_dir, vis=False) # 분석 방향에 따라 vis=True 설정
        new_row = {
            'Experiment Time': ex_time,
            'Iteration': iteration,
            'Submission Function': submission_function,
            'IoU': stats['IoU']['avg'],
            'Dice': stats['Dice']['avg'],
            'Precision': stats['Precision']['avg'],
            'Recall': stats['Recall']['avg'],
            'Output Json Pa th': output_json_path,
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)
        results_df.to_csv(csv_filename, index=False)

✅ CBAM registered: True
New https://pypi.org/project/ultralytics/8.3.114 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.10.16 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
engine\trainer: task=detect, mode=train, model=yolov8_1n.yaml, data=Datasets/Crown Detection/data_iter_01.yaml, epochs=1, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=tmp, name=250423_112117_yolov8n_Crown Detection_Iter_1, exist_ok=True, pretrained=False, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, 

train: Scanning Datasets\Crown Detection\labels... 348 images, 68 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<00:00, 1844.25it/s]

train: New cache created: Datasets\Crown Detection\labels.cache



val: Scanning Datasets\Crown Detection\labels... 116 images, 27 backgrounds, 0 corrupt: 100%|██████████| 116/116 [00:00<00:00, 1738.47it/s]

val: New cache created: Datasets\Crown Detection\labels.cache


...

train: New cache created: Datasets\Crown Detection\labels.cache


val: Scanning Datasets\Crown Detection\labels... 116 images, 24 backgrounds, 0 corrupt: 100%|██████████| 116/116 [00:00<00:00, 2150.64it/s]

val: New cache created: Datasets\Crown Detection\labels.cache
Plotting labels to tmp\250423_113643_yolov11n_Crown Detection_Iter_2\labels.jpg... 


optimizer: AdamW(lr=0.001, momentum=0.0001) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0001), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to tmp\250423_113643_yolov11n_Crown Detection_Iter_2
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      3.626       3.23      3.562        122        640:  77%|███████▋  | 17/22 [01:52<00:24,  4.95s/it]

In [ ]:
# 분석 방향에 따라 전체 결과를 모아 평균-표준편차-통계테스트 결과 저장 

keep_columns = ['Iteration','Submission Function']
keep_measures = ['IoU','Dice','Precision','Recall']  

reduction = 'Iteration'
row = 'Measure Type'
column = 'Submission Function'
reference_column = 'submission_1_20214173'  # 통계 테스트 기준이 되는 submission function

custom_fmt_template = '{mean_fmt} ± {std_fmt} {significance}'
significance_levels = [0.1, 0.05, 0.01]
decimal_places = 3
transpose = True
make_tables(csv_filename, keep_columns, keep_measures, reduction, row, column,  
            reference_column, custom_fmt_template, significance_levels, decimal_places, transpose)

FileNotFoundError: [Errno 2] No such file or directory: 'Evaluation_Results_250423_104555.csv'